In [1]:
import json
import requests
import os
import time
from datetime import datetime, timedelta

In [2]:
!git clone https://github.com/Rohan-J123/PyroPredictorModel

Cloning into 'PyroPredictorModel'...
remote: Enumerating objects: 244, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 244 (delta 198), reused 224 (delta 181), pack-reused 0 (from 0)
Receiving objects: 100% (244/244), 9.92 MiB | 19.93 MiB/s, done.
Resolving deltas: 100% (198/198), done.


In [8]:
file_paths = ["/content/PyroPredictorModel/Fire_Dataset_Weather/fire_batch_1.geojson",
              "/content/PyroPredictorModel/Fire_Dataset_Weather/fire_batch_2.geojson",
              "/content/PyroPredictorModel/Fire_Dataset_Weather/fire_batch_3.geojson",
              "/content/PyroPredictorModel/Fire_Dataset_Weather/fire_batch_4.geojson",
              "/content/PyroPredictorModel/Fire_Dataset_Weather/fire_batch_5.geojson",
              "/content/PyroPredictorModel/Fire_Dataset_Weather/fire_batch_6.geojson",
              "/content/PyroPredictorModel/Fire_Dataset_Weather/fire_batch_7.geojson",
              "/content/PyroPredictorModel/Fire_Dataset_Weather/fire_batch_8.geojson",
              "/content/PyroPredictorModel/Fire_Dataset_Weather/fire_batch_9.geojson",
              "/content/PyroPredictorModel/Fire_Dataset_Weather/fire_batch_10.geojson"]

In [9]:
output_dir = "/content/Fire_Dataset_Cleaned"
os.makedirs(output_dir, exist_ok=True)

In [10]:
i = 1
for file_path in file_paths:
    with open(file_path, 'r') as file:
        geo_data = json.load(file)
        output_data = []
        for feature in geo_data['features']:
            temp = {}
            [temp['longitude'], temp['latitude']] = feature['geometry']['coordinates']

            temp['fromdate'] = feature['properties']['fromdate']
            temp['severity'] = feature['properties']['severity']

            if feature['vegetation_indices']['NDVI'] is None:
                continue
            temp['ndvi'] = feature['vegetation_indices']['NDVI']

            if feature['vegetation_indices']['EVI'] is None:
                continue
            temp['evi'] = feature['vegetation_indices']['EVI']

            if feature['vegetation_indices']['LAI'] is None:
                continue
            temp['lai'] = feature['vegetation_indices']['LAI']

            if feature['vegetation_indices']['FPAR'] is None:
                continue
            temp['fpar'] = feature['vegetation_indices']['FPAR']

            temp['elevation'] = feature['Elevation']

            temp['relative_humidity'] = feature['Weather']['relative_humidity_2m']
            temp['dew_point'] = feature['Weather']['dew_point_2m']
            temp['surface_pressure'] = feature['Weather']['surface_pressure']
            temp['cloud_cover'] = feature['Weather']['cloud_cover']
            temp['wind_speed'] = feature['Weather']['wind_speed_10m']
            temp['soil_temperature'] = feature['Weather']['soil_temperature_0_to_7cm']
            temp['soil_moisture'] = feature['Weather']['soil_moisture_0_to_7cm']
            temp['direct_radiation'] = feature["Weather"]['direct_radiation']

            temp['weather_code'] = feature['Weather']['daily_weather_code']
            temp['temperature'] = feature['Weather']['daily_temperature_2m_mean']
            temp['precipitation'] = feature['Weather']['daily_rain_sum']
            temp['evapotranspiration'] = feature['Weather']['daily_et0_fao_evapotranspiration']
            output_data.append(temp)
        geo_data['features'] = output_data
    with open(f"/content/Fire_Dataset_Cleaned/fire_batch_{i}.geojson", 'w') as out_file:
          json.dump(geo_data, out_file, indent=4)
    i += 1

In [11]:
!zip -r Fire_Dataset_Cleaned.zip Fire_Dataset_Cleaned

  adding: Fire_Dataset_Cleaned/ (stored 0%)
  adding: Fire_Dataset_Cleaned/fire_batch_3.geojson (deflated 82%)
  adding: Fire_Dataset_Cleaned/fire_batch_8.geojson (deflated 82%)
  adding: Fire_Dataset_Cleaned/fire_batch_5.geojson (deflated 82%)
  adding: Fire_Dataset_Cleaned/fire_batch_2.geojson (deflated 83%)
  adding: Fire_Dataset_Cleaned/fire_batch_7.geojson (deflated 82%)
  adding: Fire_Dataset_Cleaned/fire_batch_1.geojson (deflated 82%)
  adding: Fire_Dataset_Cleaned/fire_batch_9.geojson (deflated 82%)
  adding: Fire_Dataset_Cleaned/fire_batch_10.geojson (deflated 81%)
  adding: Fire_Dataset_Cleaned/fire_batch_4.geojson (deflated 82%)
  adding: Fire_Dataset_Cleaned/fire_batch_6.geojson (deflated 82%)


In [15]:
file_paths = ["/content/Non_Fire_Dataset_Cleaned/non_fire_batch_1.geojson",
              "/content/Non_Fire_Dataset_Cleaned/non_fire_batch_2.geojson",
              "/content/Non_Fire_Dataset_Cleaned/non_fire_batch_3.geojson",
              "/content/Non_Fire_Dataset_Cleaned/non_fire_batch_4.geojson",
              "/content/Non_Fire_Dataset_Cleaned/non_fire_batch_5.geojson",
              "/content/Non_Fire_Dataset_Cleaned/non_fire_batch_6.geojson",
              "/content/Non_Fire_Dataset_Cleaned/non_fire_batch_7.geojson",
              "/content/Non_Fire_Dataset_Cleaned/non_fire_batch_8.geojson"]

In [16]:
import csv

data = []

for file_path in file_paths:
    with open(file_path, 'r') as file:
        data.extend(json.load(file)["features"])

with open('Non_Fires.csv', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=data[0].keys())

    # Write the header (keys of the first item in the list)
    writer.writeheader()

    # Write the rows (data entries)
    writer.writerows(data)